In [1]:
import os
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from sklearn.model_selection import train_test_split
import numpy as np


In [27]:
# !pip install scikit-learn

In [5]:
import os
from PIL import Image

# Define the path to the source folder containing your images
source_folder = r'C:\Users\user\Desktop\cv models\bottle\train\good'

# Define the path to the destination folder where resized images will be saved
destination_folder = r'C:\Users\user\Desktop\cv models\bottle\train\generated_resized'

# Ensure the destination folder exists; create it if it doesn't
image_files = [f for f in os.listdir(source_folder) if f.endswith(('jpg', 'jpeg', 'png'))]

def generator(image_files):
    datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Rescale pixel values to [0, 1]
    horizontal_flip=True,
    vertical_flip=True
    )
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # List image files in the source folder
   

    # Set the desired width and height for resizing
    width = height = 256
    desired_size = (width, height)  # Replace 'width' and 'height' with your desired dimensions

    # Loop through the image files, resize, and save them in the destination folder
    i = 0
    for image_file in image_files:
        source_path = os.path.join(source_folder, image_file)
        image = Image.open(source_path)
        image = image.resize(desired_size, Image.ANTIALIAS)  # Resize using antialiasing for better quality
        
        # Save the resized image in the destination folder
        image_array = img_to_array(image)

        # Reshape for batch processing (batch size = 1)
        image_array = image_array.reshape((1,) + image_array.shape)

        
        for batch in datagen.flow(image_array, batch_size=1):
            augmented_image = batch[0]  # Extract the augmented image
            augmented_image_path = os.path.join(destination_folder, f'augmented_{i}.jpg')
            
            # Convert the augmented image back to a valid image format and save it
            augmented_image = (augmented_image * 255).astype('uint8')  # Convert back to uint8 format
            augmented_image = array_to_img(augmented_image)  # Convert to PIL Image
            augmented_image.save(augmented_image_path)
            
            i += 1

            if i >= 1000:
                break

        # destination_path = os.path.join(destination_folder, 'resized_' + image_file)  # You can change the naming convention
        # image.save(destination_path)

generator(image_files)

# print("Resized images have been saved in the destination folder.")


C:\Users\user\AppData\Local\Temp\ipykernel_5820\4237952478.py:34: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image = image.resize(desired_size, Image.ANTIALIAS)  # Resize using antialiasing for better quality


In [25]:
# Applying Mask vgg16 to extract Features
# !pip install torch torchvision Pillow numpy



In [30]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

# Load the pre-trained VGG16 model
vgg16 = models.vgg16(pretrained=True)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\user/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [00:36<00:00, 15.1MB/s] 


In [31]:
# Extract Features from the images
import os

# Path to the folder containing training images
training_data_folder = r"C:\Users\user\Desktop\cv models\bottle\train\generated_resized"

# Path to the folder where you want to save extracted features
features_output_folder = r"C:\Users\user\Desktop\cv models\bottle\output"

# Create the features output folder if it doesn't exist
os.makedirs(features_output_folder, exist_ok=True)

# Transformation to preprocess images
transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                     std=[0.229, 0.224, 0.225])])

# Iterate through the images in the folder
for filename in os.listdir(training_data_folder):
    # if filename.endswith((".jpg", ".jpeg", ".png")):
    image_path = os.path.join(training_data_folder, filename)

    # Load and preprocess the image
    image = Image.open(image_path)
    image = transform(image)
    image = Variable(image.unsqueeze(0))  # Add a batch dimension

    # Extract features using VGG16
    features = vgg16(image)
    features = features.squeeze().detach().numpy()

    # Save the features to a file (you may want to serialize them)
    feature_output_path = os.path.join(features_output_folder, f"{os.path.splitext(filename)[0]}.npy")
    np.save(feature_output_path, features)


In [2]:
# Load extracted features
import os
import numpy as np

features_folder = r"C:\Users\user\Desktop\cv models\bottle\output"
feature_files = [os.path.join(features_folder, file) for file in os.listdir(features_folder)]

features = []
for file in feature_files:
    if file.endswith(".npy"):
        feature = np.load(file)
        features.append(feature)

features = np.array(features)


In [42]:

len(features)

1208

In [30]:
# Assuming you have a list of labels where 0 indicates seen classes and 1 indicates unseen classes
# labels = [1] * 1208 + [1] * 4
labels = [1] * 1008  + [4] * 200
labels = np.array(labels)


In [31]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Train a logistic regression classifier (replace with your preferred classifier)
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Evaluate the classifier on the test set
accuracy = classifier.score(X_test, y_test)
print("Accuracy:", accuracy)


Accuracy: 0.8099173553719008


c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [21]:
labels

array([1, 1, 1, ..., 1, 1, 1])

In [3]:
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# # Train a logistic regression classifier (replace with your preferred classifier)
# classifier = LogisticRegression()
# classifier.fit(X_train, y_train)

# # Evaluate the classifier on the test set
# accuracy = classifier.score(X_test, y_test)
# print("Accuracy:", accuracy)


In [23]:
type(labels)
labels

array([1, 1, 1, ..., 1, 1, 1])